# **Module `pepper.rle`**

✔ Revue des **typehints** et des **docstrings**.

# **`row_rle`**`(row: pd.Series) -> np.ndarray`

DEPRECATED

# **`series_rle_reduction`**`(s: pd.Series) -> np.ndarray`

## Détail des étapes

Cette fonction est complexe, nous détaillons donc ici ses étapes sur la base d'un exemple.

In [43]:
import pandas as pd
import numpy as np

s = pd.Series([1, 1, 2, 2, 2, None, None, 3, 3, 4, 4, None, 4, 4])

L'utilisation du caractère rare ☗ pour effectuer des opérations internes sur les chaînes est un classique.

Cela n'est pas absolument robuste, mais suffisamment pour la majorité des jeux de données.

In [44]:
# Replace np.nan by "☗" and cast the Series to a numpy ndarray
s = s.astype(object)
s.fillna("☗", inplace=True)
a = s.to_numpy()
display(a)

array([1.0, 1.0, 2.0, 2.0, 2.0, '☗', '☗', 3.0, 3.0, 4.0, 4.0, '☗', 4.0,
       4.0], dtype=object)

On dérive un index booléen des points d'inflexion (on repère les points où l'élément courant diffère du précédent) :

In [45]:
# Find the locations of elements that are not equal to the previous one
# The two additional `True` marks the start and the end of sequence
diff = np.concatenate(([True], a[:-1] != a[1:], [True]))
display(diff)

array([ True, False,  True, False, False,  True, False,  True, False,
        True, False,  True,  True, False,  True])

On en déduit les tailles de chacune des sections constantes successives :

In [48]:
# Calculate the count of each unique value using the diffs array
# This allows us to determine the sizes of each successive constant section
where_diff = np.where(diff)[0]
display(where_diff)
lens = np.diff(where_diff)
display(lens)

array([ 0,  2,  5,  7,  9, 11, 12, 14], dtype=int64)

array([2, 3, 2, 2, 2, 1, 2], dtype=int64)

On récupère l'ensemble sans répétition des valeurs respectives de chaque section constante :

In [49]:
# Extract the unique values from the row, including null values
vals = a[diff[:-1]]
display(vals)

array([1.0, 2.0, '☗', 3.0, 4.0, '☗', 4.0], dtype=object)

On décode les ☗ en `np.nan` :

In [50]:
# Replace "☗" by np.nan
vals[vals == "☗"] = np.nan
display(vals)

array([1.0, 2.0, nan, 3.0, 4.0, nan, 4.0], dtype=object)

On combine en paires les valeurs et leur nombre de répétitions :

In [51]:
# Create and return an array of value and count pairs
rle = np.column_stack((vals, lens))
display(rle)

array([[1.0, 2],
       [2.0, 3],
       [nan, 2],
       [3.0, 2],
       [4.0, 2],
       [nan, 1],
       [4.0, 2]], dtype=object)

# **`data_rle_reduction`**`(data: pd.DataFrame) -> pd.Series`

# **`jumps_rle`**`(s: pd.Series) -> tuple`

In [2]:
from pepper.rle import jumps_rle
import pandas as pd

s = pd.Series([0, 1, 2, 3, 4, 5])
print(jumps_rle(s))
# ((1, 6),)

s = pd.Series([0, 1, 3, 4, 5])
print(jumps_rle(s))
# ((1, 2), (2, 1), (1, 2))

s = pd.Series([0])
print(jumps_rle(s))
# ((1, 1))

s = pd.Series([], dtype=object)
print(jumps_rle(s))
# (())

((1, 6),)
((1, 2), (2, 1), (1, 2))
((1, 1),)
()


# Note à propos des arrondis et des groupby qui forcent le passage par le 64 bits

**TODO** : régler les problèmes d'arrondi dans le RLE, certainement liés à mon choix de réduction de la taille d'encodage : correction dans RLE réduction, avec l'ajout d'un paramètre de précision, qui est d'autant plus important qu'il permet de mieux synthétiser à mesure que la précision diminue.

Après analyse : c'est le `groupby` et la gestion des types non `float64` par pandas qui est en cause: les valeurs de mes groupes en `float32` ont changé à la sortie et sont des valeurs dégénérées (`0.22` est devenu `0.2199999988079071` alors que le groupement devrait se contenter de collecter les valeurs sans les modifier). La raison est la suivante : le groupby force le passage par np.float64 : en effet,`np.float4(0.22) = 0.2199999988079071` 

Les stratégies pour contourner :
1. ajouter un paramètre optionnel `decimals` à mon `series_rle_reduction` pour qu'il retraite via un `np.round` mais valeurs dégénérées par le `groupby` : c'est également une extension fonctionnelle, puisque permet de regrouper des termes proches.
2. moins intrusif et coûteux, simplement corriger le transtypage non sollicité imposé par le `groupby`, avec le passage du `dtype` d'origine à `series_rle_reduction`.
3. Avec un temps imparti réduit, rester sur du `float64` dans les tables de niveaux 2 et 3, et ne downcaster que dans le merge final.

# Expansion RLE

Du codage RLE au dataframe.

## Première partie : trame continue

### Exemple d'application

Variation mensuelle du niveau de risque de défaut client.

In [5]:
from home_credit.tables import BureauBalance

status = BureauBalance.loan_status_by_client_and_month()
display(status)

BUREAU_LOAN_STATUS_BY_CLIENT_AND_MONTH  STATUS
SK_ID_CURR MONTHS_BALANCE                     
100001     0                              0.22
           1                              0.00
           2                              0.00
           3                              0.00
           4                              0.00
...                                        ...
456255     72                             0.00
           73                             0.00
           74                             0.00
           75                             0.00
           76                             0.00

[7365660 rows x 1 columns]

### Extraction d'une série longitudinale

In [19]:
s = status.loc[100001]["STATUS"]

display(s.values)

array([0.22, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ])

### Réduction RLE de la trame

In [20]:
from pepper.rle import series_rle_reduction

r = series_rle_reduction(s)
print(r)
print(type(r))

((0.22, 1), (0.0, 51))
<class 'tuple'>


### Codage et décodage

#### Expansion directe avec `np.repeat`

In [21]:
import numpy as np
r = ((0, 1), (1, 2), (2, 3))
display(np.repeat(*(np.array(r).T)))

array([0, 1, 1, 2, 2, 2])

#### Exemple appliqué

In [6]:
r = series_rle_reduction(s)
a = np.array(r)
display(a)
values, counts = np.split(np.array(r), 2, axis=1)
counts = counts.astype(int)
print(values[:, 0])
print(counts[:, 0])
display(np.repeat(values[:, 0], counts[:, 0]))

array([[ 0.22,  1.  ],
       [ 0.  , 51.  ]])

[0.22 0.  ]
[ 1 51]


array([0.22, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ])

#### **`rle_expand_expr`**

Fonction d'expansion directe.

In [22]:
from pepper.rle import rle_expand

display(rle_expand(((0, 1), (1, 2), (2, 3))))
display(rle_expand([[1, 1], [0, 2], [.5, 1]]))

array([0, 1, 1, 2, 2, 2])

array([1. , 0. , 0. , 0.5])

#### Application (`apply`) à une suite (`pd.Series`) d'expressions

Variation mensuelle du nombre de prêts actifs d'un client.

In [1]:
from home_credit.tables import BureauBalance

activity_variation = BureauBalance.rle_loan_activity_by_client_variation(from_file=False)
display(activity_variation)

BUREAU_LOAN_ACTIVITY_BY_CLIENT_AND_MONTH_BY_CLIENT_VARIATIONS,ACTIVE
SK_ID_CURR,
100001,"[[3, 2], [2, 9], [1, 10], [2, 8], [1, 1], [0, ..."
100002,"[[1, 4], [0, 9], [1, 3], [0, 1], [1, 1], [2, 3..."
100005,"[[2, 3], [1, 10]]"
100010,"[[0, 26], [1, 10], [0, 26], [1, 10]]"
100013,"[[1, 18], [2, 22], [1, 1], [2, 3], [3, 11], [2..."
...,...
456247,"[[2, 1], [3, 2], [4, 7], [3, 1], [2, 1], [1, 1..."
456250,"[[2, 25], [3, 1], [2, 2], [1, 5]]"
456253,"[[1, 19], [4, 5], [3, 7]]"


In [6]:
from pepper.rle import rle_expand

display(activity_variation.ACTIVE.apply(rle_expand))

SK_ID_CURR
100001    [3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, ...
100002    [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...
100005              [2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
100010    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
100013    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
                                ...                        
456247    [2, 3, 3, 4, 4, 4, 4, 4, 4, 4, 3, 2, 1, 0, 0, ...
456250    [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...
456253    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
456254    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
456255    [0, 0, 0, 2, 2, 2, 2, 3, 3, 3, 4, 4, 3, 2, 2, ...
Name: ACTIVE, Length: 134542, dtype: object

## Deuxième partie : trame discontinue

### Principe

S'il semble assez facile de décoder une suite de `rle_expr` à l'aide de `rle_series.apply(rle_expand_expr)`, c'est sous l'hypothèse que la suite est continue, sans interruption.

Or, nos suivis mensuels connaissent des interruptions (des 'trous') de suivi.

Pour cette raison, et pour éviter des effets de 'creux', consistant à encoder des suites de NA, nos réductions RLE comportent deux parties :
- un **support** : la liste des indices longitudinaux où il y a effectivement suivi, codée par `jumps_rle`,
- une **trame** : la liste des valeurs correspondantes, codée par `series_rle_reduction`.

Mais j'ai fait quelque chose d'assez fin : mes expressions rle résument les cellules non NA d'un tableau à 97 dimensions.

**`rle_jumps`** code le support, c'est à dire les indices des positions successives non NA, en effectuant une réduction RLE sur différentes ('sauts') entre indices successifs, en partant (`prepend`) de -1 : par exemple `((1, 3), (3, 1), (1, 3)` correspond au support suivants : `[0, 1, 2, 5, 6, 7, 8]`.

### Problématique

Le but est de réaliser une fonction qui prend en argument deux suites, une suite de supports et une suite de trames, et qui produit le dataframe correspondant (décodé, et qui était encodé à l'aide ces deux suites).

La dimension du dataframe est celle du plus long support.

La première première étape est d'extraire cette information du codage rle des support.

Ensuite ...

Ensuite, il faut trouver la succession d'opérations la plus efficace pour réaliser l'expansion.

Pour tirer parti de la fonction précédente, basée sur `repeat`, donc efficace, je pense que la meilleure stratégie est de modifier les `rle_expr` en fonction des `rle_jumps`, pour y insérer des 'trous', c'est à dire des répétitions de `np.nan` là où le support l'indique. Enfin, on utilise notre fonction précédente avec un apply sur les `rle_expr` ainsi transformée. 

### Récupération de deux suites, de supports et de trames associées

In [3]:
from home_credit.tables import BureauBalance

supports = BureauBalance.rle_loan_tracking_period()
frames = BureauBalance.rle_loan_status_variation()
display(supports)
display(frames)

BUREAU_LOAN_TRACKING_PERIOD,MONTHS_BALANCE
SK_ID_BUREAU,
5001710,"[[1, 83]]"
5001711,"[[1, 4]]"
5001712,"[[1, 19]]"
5001713,"[[1, 22]]"
5001714,"[[1, 15]]"
...,...
6842884,"[[1, 48]]"
6842885,"[[1, 24]]"
6842886,"[[1, 33]]"


BUREAU_LOAN_STATUS_BY_MONTH_VARIATIONS,STATUS
SK_ID_BUREAU,
5001710,"[[0, 83]]"
5001711,"[[0, 4]]"
5001712,"[[0, 19]]"
5001713,"[[0, 22]]"
5001714,"[[0, 15]]"
...,...
6842884,"[[0, 48]]"
6842885,"[[5, 12], [0, 12]]"
6842886,"[[0, 33]]"


### Utilitaires d'analyse de support

#### **`rle_expr_to_numpy`**

#### **`rle_support_size`**

In [2]:
from pepper.rle import rle_support_size

print(rle_support_size(((1, 52),)))
print(rle_support_size(((3, 1), (1, 35), (18, 1), (1, 35))))
support_sizes = supports.MONTHS_BALANCE.apply(rle_support_size)
display(support_sizes)

52
72


SK_ID_BUREAU
5001710    83
5001711     4
5001712    19
5001713    22
5001714    15
           ..
6842884    48
6842885    24
6842886    33
6842887    37
6842888    62
Name: MONTHS_BALANCE, Length: 774354, dtype: int64

In [3]:
display(support_sizes[support_sizes == 0])
display(supports.loc[support_sizes == 0])

SK_ID_BUREAU
5001728    0
5001872    0
5001908    0
5001909    0
5002689    0
          ..
6841758    0
6841924    0
6841944    0
6841956    0
6842835    0
Name: MONTHS_BALANCE, Length: 5904, dtype: int64

BUREAU_LOAN_TRACKING_PERIOD,MONTHS_BALANCE
SK_ID_BUREAU,
5001728,[]
5001872,[]
5001908,[]
5001909,[]
5002689,[]
...,...
6841758,[]
6841924,[]
6841944,[]


#### **`rle_support_min_index`**

In [5]:
from pepper.rle import rle_support_min_index

print(rle_support_min_index(((1, 52),)))
print(rle_support_min_index(((3, 1), (1, 35), (18, 1), (1, 35))))
support_min_indices = supports.MONTHS_BALANCE.apply(rle_support_min_index)
display(support_min_indices)

0
2


SK_ID_BUREAU
5001710    0
5001711    0
5001712    0
5001713    0
5001714    0
          ..
6842884    0
6842885    0
6842886    0
6842887    0
6842888    0
Name: MONTHS_BALANCE, Length: 774354, dtype: int64

In [6]:
display(support_min_indices[support_min_indices > 0])
display(supports.loc[support_min_indices > 0])

SK_ID_BUREAU
5001787    1
5001788    1
5001824    1
5001859    1
5001996    1
          ..
6842558    1
6842694    1
6842696    1
6842729    1
6842771    1
Name: MONTHS_BALANCE, Length: 185112, dtype: int64

BUREAU_LOAN_TRACKING_PERIOD,MONTHS_BALANCE
SK_ID_BUREAU,
5001787,"[[2, 1], [1, 16]]"
5001788,"[[2, 1], [1, 16]]"
5001824,"[[2, 1], [1, 36]]"
5001859,"[[2, 1], [1, 33]]"
5001996,"[[2, 1], [1, 33]]"
...,...
6842558,"[[2, 1], [1, 90]]"
6842694,"[[2, 1], [1, 89]]"
6842696,"[[2, 1], [1, 84]]"


#### **`rle_support_max_index`**

In [8]:
from pepper.rle import rle_support_max_index

print(rle_support_max_index(((1, 52),)))
print(rle_support_max_index(((3, 1), (1, 35), (18, 1), (1, 35))))
support_max_indices = supports.MONTHS_BALANCE.apply(rle_support_max_index)
display(support_max_indices)

51
90


SK_ID_BUREAU
5001710    82
5001711     3
5001712    18
5001713    21
5001714    14
           ..
6842884    47
6842885    23
6842886    32
6842887    36
6842888    61
Name: MONTHS_BALANCE, Length: 774354, dtype: int64

In [10]:
display(support_max_indices[support_max_indices == 96])
display(supports.loc[support_max_indices == 96])

SK_ID_BUREAU
5011980    96
5025742    96
5028345    96
5045788    96
5047961    96
           ..
6733167    96
6746600    96
6758828    96
6801845    96
6827017    96
Name: MONTHS_BALANCE, Length: 106, dtype: int64

BUREAU_LOAN_TRACKING_PERIOD,MONTHS_BALANCE
SK_ID_BUREAU,
5011980,"[[2, 1], [1, 95]]"
5025742,"[[87, 1], [1, 10]]"
5028345,"[[66, 1], [1, 31]]"
5045788,"[[1, 97]]"
5047961,"[[52, 1], [1, 45]]"
...,...
6733167,"[[1, 97]]"
6746600,"[[37, 1], [1, 60]]"
6758828,"[[94, 1], [1, 3]]"


#### **`rle_support_expand`**

In [1]:
from pepper.rle import rle_support_expand

print(rle_support_expand(((1, 52),)))
print(rle_support_expand(((3, 1), (1, 35), (18, 1), (1, 35))))

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51]
[ 2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 35 36 37 55 56 57 58 59 60 61 62 63 64 65 66
 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90]


In [4]:
expanded_supports = supports.MONTHS_BALANCE.apply(rle_support_expand)
display(expanded_supports)

SK_ID_BUREAU
5001710    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
5001711                                         [0, 1, 2, 3]
5001712    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
5001713    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
5001714    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
                                 ...                        
6842884    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
6842885    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
6842886    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
6842887    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
6842888    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
Name: MONTHS_BALANCE, Length: 774354, dtype: object

### Nombre de colonnes du dataframe décodé

In [12]:
from pepper.rle import rle_support_max_index

max_indices = supports.MONTHS_BALANCE.apply(rle_support_max_index)
output_dim = max_indices.max() + 1
print(f"# columns: {output_dim}")

# columns: 97


In [20]:
from typing import Union
from pepper.rle import (
    rle_expr_to_numpy, rle_support_size,
    rle_support_expand, rle_expand, rle_support_max_index
)
import numpy as np

def rle_expand_row(
    rle_support_expr: Union[list, tuple, np.ndarray],
    rle_frame_expr: Union[list, tuple, np.ndarray]
) -> np.ndarray:
    """
    Decode a sequence of values encoded as an RLE (Run-Length Encoding)
    support and frame pair.


    Parameters
    ----------
    rle_support_expr : List[tuple]
        RLE sequence for support.
    rle_frame_expr : List[tuple]
        RLE sequence for frame.

    Returns
    -------
    np.ndarray
        The expanded decoded row.
    
    Examples
    --------
    >>> support = ((1, 3), (3, 1), (1, 3))
    >>> frame = ((1, 1), (2, 3), (3, 2), (4, 1))
    >>> print(f"expanded row: {rle_expand_row(support, frame)}")
    expanded row: [ 1.  2.  2. nan nan  2.  3.  3.  4.]
    >>> print(f"expanded row: {rle_expand_row([[]], [[]])}")
    expanded row: []
    """
    # Convert inputs to NumPy arrays if they are not already
    support = rle_expr_to_numpy(rle_support_expr)
    frame = rle_expr_to_numpy(rle_frame_expr)

    # Ensure that both support and frame have the same length
    if rle_support_size(support) != rle_support_size(frame):
        raise ValueError(
            "Sizes mismatch between the RLE support and the RLE frame."
        )

    if support.size == 0:
        # If the support is empty, return an empty array
        return np.array([])

    # Expand the support and frame using the appropriate functions
    expanded_support = rle_support_expand(support)
    expanded_frame = rle_expand(frame)
    
    # Calculate the maximum index in the support
    support_max_index = rle_support_max_index(support)
    
    # Expand a row filled with NaN values up to the support's maximum index
    row = rle_expand(((np.nan, support_max_index+1),))
    
    # Replace NaN values with the expanded frame values at the corresponding support indices
    row[expanded_support] = expanded_frame
    return row


support = ((1, 3), (3, 1), (1, 3))
frame = ((1, 1), (2, 3), (3, 2), (4, 1))
print(f"expanded row: {rle_expand_row(support, frame)}")
print(f"expanded row: {rle_expand_row([[]], [[]])}")

expanded row: [ 1.  2.  2. nan nan  2.  3.  3.  4.]
expanded row: []


## **`rle_expand_row`**

L'expansion de la trame donne les valeurs et celle du support les index.

Pour créer la ligne développée, il faut:
1. créer un tableau de NA de la bonne taille,
2. y affecter les valeurs de la trame développée aux indices du support développé.

In [1]:
from pepper.rle import rle_expand_row

# Test examples
support = ((1, 3), (3, 1), (1, 3))
frame = ((1, 1), (2, 3), (3, 2), (4, 1))
print(f"expanded row: {rle_expand_row(support, frame)}")
print(f"expanded row: {rle_expand_row(support, frame, row_size=10)}")
print(f"expanded row: {rle_expand_row(support, frame, row_size=8)}")
print(f"expanded row: {rle_expand_row(support, frame, row_size=0)}")

expanded row: [ 1.  2.  2. nan nan  2.  3.  3.  4.]
expanded row: [ 1.  2.  2. nan nan  2.  3.  3.  4. nan]
expanded row: [ 1.  2.  2. nan nan  2.  3.  3.]
expanded row: []


C:\Users\franc\Projects\pepper_credit_scoring_tool\python\pepper\rle.py:583: UserWarning: Row size is less than the extension of the support. Some data may be lost.
  warnings.warn(


In [2]:
from pepper.rle import rle_expand_row
import numpy as np

print(f"expanded row: {rle_expand_row(np.nan, np.nan, row_size=5)}")

expanded row: [nan nan nan nan nan]


## **`rle_expand_dataframe`**

In [1]:
from pepper.rle import rle_expand_dataframe

In [3]:
import pandas as pd

support_series = pd.Series([
    [(1, 3), (3, 1), (1, 3)],
    [(1, 2), (2, 1), (1, 2)]
])
frame_series = pd.Series([
    [(1, 1), (2, 3), (3, 2), (4, 1)],
    [(5, 1), (6, 2), (7, 2)]
])
expanded_df = rle_expand_dataframe(support_series, frame_series)
print(expanded_df)

     0    1    2    3    4    5    6    7    8
0  1.0  2.0  2.0  NaN  NaN  2.0  3.0  3.0  4.0
1  5.0  6.0  NaN  6.0  7.0  7.0  NaN  NaN  NaN


In [4]:
from home_credit.tables import BureauBalance
import pandas as pd

supports = BureauBalance.rle_loan_tracking_period()
frames = BureauBalance.rle_loan_status_variation()

display(pd.concat([supports, frames], axis=1))

,MONTHS_BALANCE,STATUS
SK_ID_BUREAU,,
5001710,"[[1, 83]]","[[0, 83]]"
5001711,"[[1, 4]]","[[0, 4]]"
5001712,"[[1, 19]]","[[0, 19]]"
5001713,"[[1, 22]]","[[0, 22]]"
5001714,"[[1, 15]]","[[0, 15]]"
...,...,...
6842884,"[[1, 48]]","[[0, 48]]"
6842885,"[[1, 24]]","[[5, 12], [0, 12]]"
6842886,"[[1, 33]]","[[0, 33]]"


In [5]:
exp_rle = rle_expand_dataframe(supports, frames)
display(exp_rle)

,0,1,2,3,4,5,6,7,8,9,...,87,88,89,90,91,92,93,94,95,96
SK_ID_BUREAU,,,,,,,,,,,,,,,,,,,,,
5001710,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5001711,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5001712,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5001713,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5001714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6842884,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6842885,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6842886,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
print(list(exp_rle.loc[6842885]))

[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]


# Future : calcul de trames RLE

Calcul de supports et de trames sans passer par la version décodée.

Calcul basé sur la représentation RLE, par exemple d'une somme de deux suites, leur intersection etc, pourra donner plus de fil à retordre, mais c'est un problème plus avancé qui n'est pas à l'ordre du jour.

## Old à recycler : problématique et principe de la solution

V1 : dépassée en contexte, mais pas en prolongeant vers le calcul de codages RLE (somme, intersection, etc de séquences ainsi codées).

**TODO** ajouter des exemples concrets.

Avec un support $(1, N)$, il n'y a rien à faire, séquence continue, pas de trous.

L'opération de transformation de la trame RLE est l'identité (c'est le cas le plus fréquent en la circonstance de nos données Home Credit).

Avec un support fragmenté, de la forme $(1, i)(k, j)(1, l)$, il y a un trou de taille $k-1$ après la séquence des $i$ premières valeurs, suivi des $l+1$ valeurs suivantes.

Avec $k_j > 1$,

Avec un support $(1, i_1)(k_1, j_1)(1, i_2)(k_2, j_2)\ldots(1, i_n)(k_n, j_n)(1, i_{n+1})$, il y a $n$ 'trous' de tailles $k_1-1, \dots, k_n-1$, précédés de $i_1, i_2+1, i_3+1, \ldots, i_n+1$ valeurs.

Attention au cas où cela démarre par un trou !

Cela revient donc à insérer $n$ `(np.nan, k_j-1)` dans l'expression RLE des valeurs.

Là il faut réfléchir un peu pour le faire en Numpy.

Les distances d'insertion sont données par les $i_1, i_2+1, i_3+1, \ldots, i_n+1$.

Par exemple, pour la première insertion, on identifie dans la séquence de valeurs, l'élément répété, tel que $i_1$ soit supérieur strictement à la somme des répétitions des éléments précédents, mais inférieure ou égale à cette somme augmentée du nombre de répétitions de cet élément.

Là, il faut éventuellement fragmenter l'élément en deux parties, celle qui précède le trou et celle qui le suit.

On procède à l'insertion.

La suite et une répétition de ce procédé.

V2 :

Support : $s = (j_1, k_1)\ldots(j_n, k_n)$.

#### **`rle_support_hole_count`**

Pour le calcul de trames.

In [4]:
from pepper.rle import rle_support_hole_count

print(rle_support_hole_count(((1, 1),)))
print(rle_support_hole_count(((2, 1),)))
print(rle_support_hole_count(((2, 2),)))
print(rle_support_hole_count(((2, 1), (1, 10), (3, 2))))

0
1
2
3


### Principe de la solution

In [2]:
from pepper.rle import (
    rle_expr_to_numpy,
    rle_expand, rle_support_expand,
    rle_support_size, rle_support_max_index
)

import numpy as np

support = ((1, 3), (3, 1), (1, 3))
frame = ((1, 1), (2, 3), (3, 2), (4, 1))

rle_support = rle_expr_to_numpy(support)
rle_frame = rle_expr_to_numpy(frame)

print(f"support: {support}")
print(f"frame: {frame}")

expanded_support = rle_support_expand(support)
expanded_frame = rle_expand(frame)
print(f"expanded support: {expanded_support}")
print(f"expanded frame: {expanded_frame}")

support_size = rle_support_size(support)
print(f"support size: {support_size}")

support_max_index = rle_support_max_index(support)
print(f"support max index: {support_max_index}")

row = rle_expand(((np.nan, support_max_index+1),))
row[expanded_support] = expanded_frame

print(f"expanded row: {row}")

support: ((1, 3), (3, 1), (1, 3))
frame: ((1, 1), (2, 3), (3, 2), (4, 1))
expanded support: [0 1 2 5 6 7 8]
expanded frame: [1 2 2 2 3 3 4]
support size: 7
support max index: 8
expanded row: [ 1.  2.  2. nan nan  2.  3.  3.  4.]


Analyse des 'trous' :

In [3]:
from pepper.rle import rle_support_hole_count

print(rle_support_hole_count(rle_support))

hole_sizes = rle_support[rle_support[:, 0] > 1, 0] - 1
print(f"hole sizes (k-1): {hole_sizes}")

start_with_hole = rle_support[0, 0] > 1
print(f"start with hole: {start_with_hole}")

seq_sizes = rle_support[rle_support[:, 0] == 1, 0] + 1
if not start_with_hole:
    seq_sizes[0] -= 1
print(f"seq sizes (i+1): {seq_sizes}")

1
hole sizes (k-1): [2]
start with hole: False
seq sizes (i+1): [1 2]
